In [1]:
import pandas as pd
import numpy as np

df = pd.DataFrame()
df = pd.read_csv('Sheet_1.csv')
df.head(15)

,response_id,class,response_text,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,response_1,not_flagged,I try and avoid this sort of conflict,NaN,NaN,NaN,NaN,NaN
1,response_2,flagged,Had a friend open up to me about his mental ad...,NaN,NaN,NaN,NaN,NaN
2,response_3,flagged,I saved a girl from suicide once. She was goin...,NaN,NaN,NaN,NaN,NaN
3,response_4,not_flagged,i cant think of one really...i think i may hav...,NaN,NaN,NaN,NaN,NaN
4,response_5,not_flagged,Only really one friend who doesn't fit into th...,,NaN,NaN,NaN,NaN
5,response_6,not_flagged,a couple of years ago my friends was going to ...,NaN,NaN,NaN,NaN,NaN
6,response_7,flagged,Roommate when he was going through death and l...,NaN,NaN,NaN,NaN,NaN
7,response_8,flagged,i've had a couple of friends (you could say mo...,NaN,NaN,NaN,NaN,NaN
8,response_9,not_flagged,Listened to someone talk about relationship tr...,NaN,NaN,NaN,NaN,NaN
9,response_10,flagged,I will always listen. I comforted my sister wh...,NaN,NaN,NaN,NaN,NaN


In [4]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

desc_lines = []
lines = df['response_text'].values.tolist()

for line in lines :
    tokens = word_tokenize(line)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('','',string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    desc_lines.append(words)
    len(desc_lines)

In [25]:
import gensim

model = gensim.models.Word2Vec(sentences = desc_lines, size = 512, window = 5, workers = 6, min_count = 1)
words = list(model.wv.vocab)
print('Vocabulary size %d' %len(words))

W1116 12:10:59.312075 13356 base_any2vec.py:1386] under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay


Vocabulary size 551


In [7]:
filename = 'sheet1_embedding.txt'
model.wv.save_word2vec_format(filename, binary = False)

In [8]:
import os

embeddings_index = {}
f = open(os.path.join('', 'sheet1_embedding.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
    
f.close()

In [10]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

X = df['response_text']
Y = df['class']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 22)

max_length = max([len(s.split()) for s in X])
print(max_length)

304


In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tkn = Tokenizer()
tkn.fit_on_texts(desc_lines)
sequences = tkn.texts_to_sequences(desc_lines)

word_index = tkn.word_index
print("Found %s unique tokens" %len(word_index))

review_pad = pad_sequences(sequences, maxlen = max_length)

Using TensorFlow backend.


In [12]:
from sklearn import preprocessing
from keras.utils import to_categorical

def one_hot():
    le = preprocessing.LabelEncoder()
    le.fit(df['class'])
    #list(le.classes_))
    var = list(le.transform(df['class']))
    var = to_categorical(var)
    return var

Found 551 unique tokens


In [13]:
print("Shape of description tensor :", review_pad.shape)
print("Shape of class tensor :", one_hot().shape)

Shape of description tensor : (80, 304)
Shape of class tensor : (80, 2)


In [14]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, 512))

for word, i in word_index.items() :
    if i > num_words :
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None :
        embedding_matrix[i] = embedding_vector
        
print(num_words)

552


In [26]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, Dropout
from keras.layers.embeddings import Embedding 
from keras.initializers import Constant

model = Sequential()
embedding_layer = Embedding(num_words, 512, 
                            embeddings_initializer = Constant(embedding_matrix), 
                            input_length = max_length,
                            trainable = False
                           )
model.add(embedding_layer)
model.add(LSTM(128, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(90, return_sequences = False))
model.add(Dropout(0.5))
model.add(Dense(2, activation = 'sigmoid'))

model.compile(optimizer ='adam', loss='binary_crossentropy', metrics =['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 304, 512)          282624    
_________________________________________________________________
lstm_5 (LSTM)                (None, 304, 128)          328192    
_________________________________________________________________
dropout_4 (Dropout)          (None, 304, 128)          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 90)                78840     
_________________________________________________________________
dropout_5 (Dropout)          (None, 90)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 182       
Total params: 689,838
Trainable params: 407,214
Non-trainable params: 282,624
__________________________________________

In [27]:
validation_split = 0.3

indices = np.arange(review_pad.shape[0])
np.random.shuffle(indices)
review_pad = review_pad[indices]
num_validation_samples = int(validation_split * review_pad.shape[0])

X_train_pad = review_pad[:-num_validation_samples]
X_test_pad = review_pad[-num_validation_samples:]

print(X_test_pad.shape)

Y_train_oh = one_hot()[:-num_validation_samples]
Y_test_oh = one_hot()[-num_validation_samples:]

print(Y_test_oh.shape)

(24, 304)
(24, 2)


In [28]:
model.fit(X_train_pad, Y_train_oh, epochs = 25, batch_size = 8, shuffle=True)

Epoch 1/25
56/56 [==============================] - 7s 122ms/step - loss: 0.6884 - acc: 0.6339
Epoch 2/25
56/56 [==============================] - 5s 92ms/step - loss: 0.6586 - acc: 0.6964
Epoch 3/25
56/56 [==============================] - 5s 97ms/step - loss: 0.6549 - acc: 0.6964
Epoch 4/25
56/56 [==============================] - 6s 100ms/step - loss: 0.6318 - acc: 0.6964
Epoch 5/25
56/56 [==============================] - 5s 97ms/step - loss: 0.6090 - acc: 0.6964
Epoch 6/25
56/56 [==============================] - 5s 97ms/step - loss: 0.6347 - acc: 0.6964
Epoch 7/25
56/56 [==============================] - 6s 100ms/step - loss: 0.6222 - acc: 0.6964
Epoch 8/25
56/56 [==============================] - 6s 105ms/step - loss: 0.6277 - acc: 0.6964
Epoch 9/25
56/56 [==============================] - 6s 111ms/step - loss: 0.6177 - acc: 0.6964
Epoch 10/25
56/56 [==============================] - 6s 102ms/step - loss: 0.6170 - acc: 0.6964
Epoch 11/25
56/56 [==============================] - 